In [6]:
from cerebro.backend import SparkBackend
from cerebro.keras import SparkEstimator

# datas storage for intermediate data and model artifacts.
from cerebro.storage import LocalStore, HDFSStore

# Model selection/AutoML methods.
from cerebro.tune import GridSearch, RandomSearch, TPESearch

# Utility functions for specifying the search space.
from cerebro.tune import hp_choice, hp_uniform, hp_quniform, hp_loguniform, hp_qloguniform

import tensorflow as tf
from pyspark.sql import SparkSession
import numpy as np


spark = SparkSession \
    .builder \
    .appName("Cerebro Example") \
    .getOrCreate()

...
work_dir = '/Users/zijian/Desktop/ucsd/cse234/project/cerebro-system/'
backend = SparkBackend(spark_context=spark.sparkContext, num_workers=1)
store = LocalStore(prefix_path=work_dir + 'test/')

df = spark.read.format("libsvm") \
    .option("numFeatures", "784") \
    .load("/Users/zijian/Desktop/ucsd/cse234/project/mnist/mnist.scale") \

from pyspark.ml.feature import OneHotEncoderEstimator

encoder = OneHotEncoderEstimator(dropLast=False)
encoder.setInputCols(["label"])
encoder.setOutputCols(["label_OHE"])

encoder_model = encoder.fit(df)
encoded = encoder_model.transform(df)

feature_columns=['features']
label_columns=['label_OHE']

encoded.show()
train_df, test_df = encoded.randomSplit([0.8, 0.2], 100)

CEREBRO => Time: 2021-11-29 02:59:30, Running 1 Workers


+-----+--------------------+--------------+
|label|            features|     label_OHE|
+-----+--------------------+--------------+
|  5.0|(784,[152,153,154...|(10,[5],[1.0])|
|  0.0|(784,[127,128,129...|(10,[0],[1.0])|
|  4.0|(784,[160,161,162...|(10,[4],[1.0])|
|  1.0|(784,[158,159,160...|(10,[1],[1.0])|
|  9.0|(784,[208,209,210...|(10,[9],[1.0])|
|  2.0|(784,[155,156,157...|(10,[2],[1.0])|
|  1.0|(784,[124,125,126...|(10,[1],[1.0])|
|  3.0|(784,[151,152,153...|(10,[3],[1.0])|
|  1.0|(784,[152,153,154...|(10,[1],[1.0])|
|  4.0|(784,[134,135,161...|(10,[4],[1.0])|
|  3.0|(784,[123,124,125...|(10,[3],[1.0])|
|  5.0|(784,[216,217,218...|(10,[5],[1.0])|
|  3.0|(784,[143,144,145...|(10,[3],[1.0])|
|  6.0|(784,[72,73,74,99...|(10,[6],[1.0])|
|  1.0|(784,[151,152,153...|(10,[1],[1.0])|
|  7.0|(784,[211,212,213...|(10,[7],[1.0])|
|  2.0|(784,[151,152,153...|(10,[2],[1.0])|
|  8.0|(784,[159,160,161...|(10,[8],[1.0])|
|  6.0|(784,[100,101,102...|(10,[6],[1.0])|
|  9.0|(784,[209,210,211...|(10,

In [1]:
import autokeras as ak
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from keras_tuner.engine import hyperparameters

num_classes = 10
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255
x_test = x_test / 255

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(x_train.shape)  # (60000, 28, 28, 1)
print(y_train.shape)  # (60000, 10)

# input_node = ak.ImageInput()
# output_node = ak.ImageBlock(
#     # Only search ResNet architectures.
#     block_type="resnet",
#     # Normalize the dataset.
#     normalize=True,
#     # Do not do data augmentation.
#     augment=False,
# )(input_node)
# output_node = ak.ClassificationHead()(output_node)

input_node = ak.ImageInput()
output_node = ak.ConvBlock(
    kernel_size=hyperparameters.Fixed('kernel_size', value=3),
    num_blocks=hyperparameters.Fixed('num_blocks', value=1),
    num_layers=hyperparameters.Fixed('num_layers', value=3),
)(input_node)
output_node = ak.ClassificationHead()(output_node)

clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=16, seed=2000
)

(60000, 28, 28, 1)
(60000, 10)


In [2]:
dataset, validation_data = clf._convert_to_dataset(
    x=x_train, y=y_train, validation_data=None, batch_size=64
)

clf._analyze_data(dataset)
clf._build_hyper_pipeline(dataset)

clf.tuner.hyper_pipeline.outputs = [[]]

2021-11-29 07:30:07.565401: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-11-29 07:30:07.565956: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from autokeras.utils import data_utils
dataset, validation_data = data_utils.split_dataset(
    dataset, 0.2
)
history = clf.tuner.search(
    x=dataset,
    epochs=1,
    callbacks=None,
    validation_data=validation_data,
    validation_split=0.2,
    verbose=1,
)

Trial 1 Complete [00h 00m 53s]
val_loss: 0.6419660449028015

Best val_loss So Far: 0.6419660449028015
Total elapsed time: 00h 00m 53s
INFO:tensorflow:Oracle triggered exit
938/938 [==============================] - 64s 68ms/step - loss: 1.7050 - accuracy: 0.4011


2021-11-29 07:07:04.227290: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


In [29]:
trial_rels = {}
for tid in clf.tuner.oracle.trials:
    m = clf.tuner.oracle.trials[tid].get_state()['metrics']['metrics']
    trial_rels[tid] = {}
    trial_rels[tid]['val_loss'] = m['val_loss']['observations']
    trial_rels[tid]['val_accuracy'] = m['val_accuracy']['observations']

trial_rels

{'e46dfdf2c7c0369943e1fcf59cc72118': {'val_loss': [{'value': [0.6419660449028015],
    'step': 0}],
  'val_accuracy': [{'value': [0.8532171845436096], 'step': 0}]}}

In [18]:
clf.tuner.results_summary()

Results summary
Results in ./auto_model
Showing 10 best trials
Objective(name='val_loss', direction='min')
Trial summary
Hyperparameters:
conv_block_1/kernel_size: 3
conv_block_1/separable: False
conv_block_1/max_pooling: True
conv_block_1/dropout: 0
conv_block_1/num_blocks: 1
conv_block_1/num_layers: 3
conv_block_1/filters_0_0: 32
conv_block_1/filters_0_1: 32
conv_block_1/filters_0_2: 32
classification_head_1/spatial_reduction_1/reduction_type: global_max
classification_head_1/dropout: 0.5
optimizer: adam
learning_rate: 0.001
Score: 0.6419660449028015


In [10]:
model = clf.tuner.get_best_model()

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
cast_to_float32 (CastToFloat (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 22, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 11, 11, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 3872)              0     

In [12]:
trial = clf.tuner.oracle.get_best_trials(1)[0]
hps = trial.hyperparameters

In [13]:
hps.values

{'conv_block_1/kernel_size': 3,
 'conv_block_1/separable': False,
 'conv_block_1/max_pooling': True,
 'conv_block_1/dropout': 0,
 'conv_block_1/num_blocks': 1,
 'conv_block_1/num_layers': 3,
 'conv_block_1/filters_0_0': 32,
 'conv_block_1/filters_0_1': 32,
 'conv_block_1/filters_0_2': 32,
 'classification_head_1/spatial_reduction_1/reduction_type': 'flatten',
 'classification_head_1/dropout': 0.5,
 'optimizer': 'adam',
 'learning_rate': 0.001}

In [14]:
clf.tuner.hyper_pipeline.inputs

[[]]

In [16]:
clf.tuner.hyper_pipeline.outputs

[[]]

In [18]:
clf.tuner.hypermodel.hypermodel._get_loss()

{'classification_head_1': <tensorflow.python.keras.losses.CategoricalCrossentropy at 0x17ec71e10>}

In [5]:
rel = np.random.rand(16,5,2)
rel

array([[[0.57051729, 0.56452876],
        [0.48844183, 0.33647775],
        [0.37586818, 0.53203587],
        [0.06810629, 0.58452906],
        [0.23789776, 0.16075658]],

       [[0.15211915, 0.12706922],
        [0.32744117, 0.69415387],
        [0.35896647, 0.89136566],
        [0.20187988, 0.1106025 ],
        [0.52755676, 0.86765533]],

       [[0.26187438, 0.49939512],
        [0.4285644 , 0.53000389],
        [0.57394682, 0.39185565],
        [0.07951412, 0.86728101],
        [0.16450358, 0.80348162]],

       [[0.28792825, 0.26857118],
        [0.71747802, 0.88140566],
        [0.99212003, 0.45835004],
        [0.78512927, 0.73869394],
        [0.47358298, 0.48483471]],

       [[0.9330051 , 0.75265384],
        [0.15928883, 0.13481047],
        [0.41067717, 0.75653626],
        [0.21251302, 0.3140294 ],
        [0.20127344, 0.9384252 ]],

       [[0.50441484, 0.60373145],
        [0.83754858, 0.61018246],
        [0.73557276, 0.40976613],
        [0.48964001, 0.50892452],
    

In [7]:
rel = rel.reshape(4,4,5,2)
rel

array([[[[0.57051729, 0.56452876],
         [0.48844183, 0.33647775],
         [0.37586818, 0.53203587],
         [0.06810629, 0.58452906],
         [0.23789776, 0.16075658]],

        [[0.15211915, 0.12706922],
         [0.32744117, 0.69415387],
         [0.35896647, 0.89136566],
         [0.20187988, 0.1106025 ],
         [0.52755676, 0.86765533]],

        [[0.26187438, 0.49939512],
         [0.4285644 , 0.53000389],
         [0.57394682, 0.39185565],
         [0.07951412, 0.86728101],
         [0.16450358, 0.80348162]],

        [[0.28792825, 0.26857118],
         [0.71747802, 0.88140566],
         [0.99212003, 0.45835004],
         [0.78512927, 0.73869394],
         [0.47358298, 0.48483471]]],


       [[[0.9330051 , 0.75265384],
         [0.15928883, 0.13481047],
         [0.41067717, 0.75653626],
         [0.21251302, 0.3140294 ],
         [0.20127344, 0.9384252 ]],

        [[0.50441484, 0.60373145],
         [0.83754858, 0.61018246],
         [0.73557276, 0.40976613],
        

In [10]:
np.min(rel[...,0], axis=1)

array([[0.15211915, 0.32744117, 0.35896647, 0.06810629, 0.16450358],
       [0.50441484, 0.02130739, 0.37456516, 0.21251302, 0.20127344],
       [0.76403764, 0.17500909, 0.2626636 , 0.12499406, 0.12832699],
       [0.27764773, 0.46477077, 0.53153129, 0.00879266, 0.2866647 ]])

In [11]:
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
from autokeras.utils import data_utils

dataset, validation_data = data_utils.split_dataset(
    dataset, 0.2
)
history = clf.tuner.search(
    x=dataset,
    epochs=5,
    callbacks=None,
    validation_data=validation_data,
    validation_split=0.2,
    verbose=1,
)
tuner = clf.tuner
tuner.hypermodel.hypermodel.set_fit_args(validation_split, epochs=epochs)
hp = tuner.oracle.get_space()
tuner.hypermodel.hypermodel.set_io_shapes(data_utils.dataset_shape(dataset))
tuner.hypermodel.build(hp)
tuner.oracle.update_space(hp)

trial_rels = []
i = 0
max_trial = 16
ts = tuner.oracle.create_trials(max_trial, tuner.tuner_id)
while i < max_trial:
    trial = ts[i]
    for lr in [0.01, 0.001]:
        for bs in [64, 128]:
            if i < max_trial:
                tuner.hypermodel.hypermodel.set_io_shapes(data_utils.dataset_shape(dataset))
                model = tuner.hypermodel.build(trial.hyperparameters)
                tuner.adapt(model, dataset)
                loss = 'categorical_crossentropy'
                optimizer = tf.keras.optimizers.Adam(lr=lr)
                model.compile(loss=loss, optimizer=optimizer, metrics=["accuracy"])
                hist = model.fit(x_train, y_train, batch_size=bs, epochs=5, validation_split=0.2)
                trial_rels.append(hist)
                print(trial.trial_id + str(i))
                i = i + 1